In [1]:
import cv2
import numpy as np

images=[]
images_grey=[]

for i in range(51):
    images.append(cv2.imread(f"C:\\Users\\karan\\hotelImages\\hotel.seq{i}.png"))
    images_grey.append(cv2.imread(f"C:\\Users\\karan\\hotelImages\\hotel.seq{i}.png",0))
images = np.asarray(images)
images_grey = np.asarray(images_grey)


def detectKeypoints (currImage, tau):
    img = np.float64(cv2.cvtColor(currImage,cv2.COLOR_BGR2GRAY))
    Ix = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
    Iy = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
    Ixx = np.multiply(Ix,Ix)
    Iyy = np.multiply(Iy,Iy)
    Ixy = np.multiply(Ix,Iy)
    Iyx = np.multiply(Ix,Iy)
    Igx = cv2.GaussianBlur(Ixx,(5,5),1)
    Igy = cv2.GaussianBlur(Iyy,(5,5),1)
    Igxy = cv2.GaussianBlur(Ixy,(5,5),1)
    Igyx = cv2.GaussianBlur(Iyx,(5,5),1)
    det = np.multiply(Igx,Igy) - np.multiply(Igxy,Igyx)
    trace =  np.multiply((Igx + Igy),(Igx + Igy))
    Q = det-0.04*(trace)
#     Alpha = 0.04
    final_image,corners = show_corners(Q,img,currImage,tau)
    return corners

def show_corners(corners, img,currImage,tau):
    image = currImage.copy()
    corners = non_maximum_suppresor(corners,img,tau)
    for i in corners:
        x,y = i.ravel()
        image[x][y] = [0,255,0]
    cv2.imshow("Corner", image)
    return image,corners

def non_maximum_suppresor(image, img,tau):
    result = np.zeros((len(img), len(img[0])))
    for i in range(1,len(img) - 1):
        for j in range(1, len(img[0]) - 1):
            if image[i, j] > image[i-1, j-1] and image[i, j] > image[i-1, j+1]\
            and image[i, j] > image[i+1, j-1] and image[i, j] > image[i+1, j+1]\
            and image[i, j] > image[i+1, j] and image[i-1, j] > image[i+1, j+1]\
            and image[i, j] > image[i, j+1] and image[i, j] > image[i, j-1]:
                result[i, j] = image[i, j]
            else:
                result[i, j] = 0
    xmax, xmin = result.max(), result.min()
    result = (result - xmin)/(xmax - xmin)
    HCResponseTreshold = (result >= tau)
    candidates = np.transpose(HCResponseTreshold.nonzero())    
    return candidates

def trackPoints(movedOutFlag, pts, im, ws):
    #% Tracking initial points (pt_x, pt_y) across the image sequence
    #% track_x: [Number of keypoints] x [2]
    #% track_y: [Number of keypoints] x [2]
    N = pts.shape[0]
    nim = im.shape[0]    
    trackPts = np.zeros( [nim, N, 2], dtype=float)
    trackPts[0] = pts[:, 0:2]
    for t in range(nim-1):
        ( movedOutFlag, trackPts[t+1,:] ) = \
        getNextPoints(movedOutFlag, trackPts[t, :], im[t], im[t+1], ws)
    return (movedOutFlag, trackPts)
    
def getNextPoints(movedOutFlag, results, im1, im2, ws):
    #% Iterative Lucas-Kanade feature tracking
    #% x, y : initialized keypoint position in im2
    #% x2, y2: tracked keypoint positions in im2
    #% ws: patch window size
    xy2 = results
    I = np.float32(im1) 
    Ix = cv2.Sobel(I, cv2.CV_32F, 1, 0, ksize=3)
    Iy = cv2.Sobel(I, cv2.CV_32F, 0, 1, ksize=3)
    Ixy = np.multiply(Ix, Iy)
    halfSide = np.int32(np.floor(ws/2))
    numRows = results.shape[0]
    for i in range(xy2.shape[0]):
        if movedOutFlag[i] == 0:
            patch1 = (cv2.getRectSubPix(I,(15,15),(xy2[i,1],xy2[i,0])))
            I_x = cv2.getRectSubPix(Ix,(15,15),(xy2[i,1],xy2[i,0]))
            I_y = cv2.getRectSubPix(Iy,(15,15),(xy2[i,1],xy2[i,0]))
            I_xy = cv2.getRectSubPix(Ixy,(15,15),(xy2[i,1],xy2[i,0]))
            for _ in range(10):
                patch2 = cv2.getRectSubPix(im2,(15,15),(xy2[i,1],xy2[i,0]))
                It = patch2-patch1
                
                M = np.array([[np.sum(np.square(I_x)),np.sum(I_xy)],[np.sum(I_xy),np.sum(np.square(I_y))]])
                a = np.array([np.sum(I_x*It), np.sum(I_y*It)])
                try:
                    s = np.linalg.solve(M,-a)
                except:
                    pass
                if xy2[i,1] >im1.shape[1] or xy2[i,0] > im1.shape[0]:
                    movedOutFlag[i] = 1
                else:
                    xy2[i,0] += s[1]
                    xy2[i,1] += s[0]         
    return ( movedOutFlag, xy2 )
RED = (0, 0, 255)
GREEN = (0, 255, 0)
BLUE = (255, 0, 0)
tau = 0.05 # threshold for Harris corner detection
imagesInFolder = 50
currImage = images[0]
results = detectKeypoints(currImage, tau)
curr_grey = images_grey[0]
movedOutFlag = np.zeros(results.shape[0]); # flags for starting points
patchSize = 15 # Tracking patchSize by patchSize patches
(movedOutFlag, trackXY) = trackPoints(movedOutFlag, results,images_grey, patchSize)
image = cv2.cvtColor(curr_grey, cv2.COLOR_GRAY2BGR)
for result in (results):
    cv2.circle(image, (np.uint32(result[0]), np.uint32(result[1])), 1, GREEN)
for ptIndex in range(200):
    if (movedOutFlag[ptIndex] == 0):
        for loc in (trackXY[:,ptIndex,:]):
            cv2.circle(image, (np.uint32(loc[0]), np.uint32(loc[1])), 1, RED)
cv2.imshow('track',image)
cv2.waitKey(0)

-1